In [72]:
!mkdir FOOTBALL_VS_CRICKET
!aws s3 cp s3://gamebot-2022-training-data/Detection_FootballvsCricketBall.zip ./FOOTBALL_VS_CRICKET
!unzip ./FOOTBALL_VS_CRICKET/Detection_FootballvsCricketBall.zip -d ./FOOTBALL_VS_CRICKET

!ls images/*.jpg| tr '\n' '\n'>images.txt

In [73]:
import pandas as pd
import numpy as np


images_df = pd.read_csv("images.txt", sep=" ", names=['filename'])
images_df.sort_values(by=['filename'])
images_df.reset_index()
images_df['index'] = images_df.index + 1
flipped_images_df = images_df[['index','filename']]
flipped_images_df.sort_values(by=['index'])
flipped_images_df['filename'] = flipped_images_df['filename'].str.replace('images/', '')
flipped_images_df.to_csv('images.txt', sep=' ', index=False, header=False)




flipped_images_df

,index,filename
0,1,0027773a6d54b960.jpg
1,2,002af65597d93000.jpg
2,3,004130acea29204f.jpg
3,4,00490dffc1ce9a88.jpg
4,5,00ac400078dcfb49.jpg
...,...,...
864,865,ff59efc44f003e17.jpg
865,866,ff8ac28dbdbdb760.jpg
866,867,ffcd8c6fc8232136.jpg
867,868,ffd627061d34ee2e.jpg


In [128]:
import csv
valid_labels_list_df = pd.read_csv("valid_labels_list.txt", sep=" ", names=['filename'])
valid_labels_list_df['filename'] = valid_labels_list_df['filename'].astype(str)
valid_labels_list_df['filename'] = valid_labels_list_df['filename'].str.replace('valid/', '')
valid_labels_list_df['filename'] = valid_labels_list_df['filename'].str.replace('txt', 'jpg')
valid_labels_list_df['is_training']= '0'

train_labels_list_df = pd.read_csv("train_labels_list.txt", sep=" ", names=['filename'])
train_labels_list_df['filename'] = train_labels_list_df['filename'].astype(str)
train_labels_list_df['filename'] = train_labels_list_df['filename'].str.replace('train/', '')
train_labels_list_df['filename'] = train_labels_list_df['filename'].str.replace('txt', 'jpg')
train_labels_list_df['is_training']= '1'

combined_labels_list_df = pd.concat([train_labels_list_df,valid_labels_list_df],axis=0, ignore_index=True)

combined_labels_list_df = pd.merge(flipped_images_df, combined_labels_list_df, on=["filename"])
#combined_labels_list_df[['index', 'is_training']].to_csv('train_test_split.txt', sep=' ', index=False, header=False)

def get_label(img_filename, is_training):
    text_filename = img_filename.replace('.jpg', '.txt')
    if is_training == '0':
        folder = 'valid'
    else:
        folder = 'train'
    
    with open('labels/'+folder+'/'+text_filename, 'r') as f:
        #print('labels/'+folder+'/'+text_filename)
        data = f.read()
        parts = data.split(' ')
        label = parts[0]
        if label == '1':
            return '2'
        else:
            return '1'


def get_box(img_filename, is_training):
    text_filename = img_filename.replace('.jpg', '.txt')
    if is_training == '0':
        folder = 'valid'
    else:
        folder = 'train'
    
    with open('labels/'+folder+'/'+text_filename, 'r') as f:
        #print('labels/'+folder+'/'+text_filename)
        data = f.read()
        parts = data.split(' ')
        label = parts[0]
        return " ".join(parts[1:])
        
combined_labels_list_df['label'] = combined_labels_list_df.apply(
    lambda row: get_label(row['filename'], row['is_training']), 
    axis=1
)

combined_labels_list_df['bounding_box'] = combined_labels_list_df.apply(
    lambda row: get_box(row['filename'], row['is_training']), 
    axis=1
)

#combined_labels_list_df[['label', 'x', 'y', 'w', 'h']] = combined_labels_list_df['label_data'].str.split(' ', expand=True)
#combined_labels_list_df['label_data'].str.split(' ')

#combined_labels_list_df[['index', 'label']].to_csv('image_class_labels.txt', sep=' ', index=False, header=False)
combined_labels_list_df[['index', 'bounding_box']].to_csv('bounding_boxes.txt', sep=' ', index=False, header=False)
#TODO: Fix issue with multiple bounding boxes in one image

combined_labels_list_df
#test

,index,filename,is_training,label,bounding_box
0,1,0027773a6d54b960.jpg,0,2,0.208496 0.461650 0.020508 0.026049
1,2,002af65597d93000.jpg,0,2,0.545410 0.109233 0.061523 0.078023
2,3,004130acea29204f.jpg,0,2,0.360374 0.748047 0.242871 0.321615
3,4,00490dffc1ce9a88.jpg,0,2,0.644163 0.107422 0.132208 0.095703
4,5,00ac400078dcfb49.jpg,1,1,0.031738 0.238620 0.012695 0.016153
...,...,...,...,...,...
864,865,ff59efc44f003e17.jpg,0,2,0.585608 0.913086 0.327543 0.125000
865,866,ff8ac28dbdbdb760.jpg,0,1,0.737305 0.432718 0.027344 0.036939
866,867,ffcd8c6fc8232136.jpg,0,1,0.253418 0.562408 0.018555 0.026432
867,868,ffd627061d34ee2e.jpg,1,2,0.694002 0.905762 0.195838 0.151367


In [34]:
images_df = pd.read_csv("images.txt", sep=" ", names=['index', 'filename'])
images_df

,index,filename
0,1,0027773a6d54b960.jpg
1,2,002af65597d93000.jpg
2,3,004130acea29204f.jpg
3,4,00490dffc1ce9a88.jpg
4,5,00ac400078dcfb49.jpg
...,...,...
864,865,ff59efc44f003e17.jpg
865,866,ff8ac28dbdbdb760.jpg
866,867,ffcd8c6fc8232136.jpg
867,868,ffd627061d34ee2e.jpg
